In [1]:
import pandas as pd
import duckdb as db


movie = pd.read_csv('../Dataset/movie_activity.csv')

movie.head()

,user_id,movie_name,activity_date,status,watch_time_minutes,country_code
0,1,Avengers: Endgame,2025-04-10,started,52,US
1,1,Inside Out,2024-11-10,started,42,US
2,1,Knives Out,2024-10-27,started,6,US
3,1,Turning Red,2024-12-13,started,19,US
4,1,Spider-Man: No Way Home,2025-03-22,started,6,US


In [2]:
users = pd.read_csv('../Dataset/users_movies.csv')

users.head()

,id,created_at,country_code,first_date,first_movie,last_date,last_movie,started,finished
0,1,2024-10-16,US,2024-10-19,Soul,2025-04-26,Her,15,4
1,2,2023-10-24,US,2025-01-03,Avengers: Endgame,2025-03-03,Moonlight,18,14
2,3,2024-10-02,CA,2025-01-17,Spider-Man: No Way Home,2025-06-29,The Shawshank Redemption,10,5
3,4,2023-03-07,CA,2023-03-28,Inside Out,2024-11-19,Her,38,14
4,5,2023-10-06,CA,2024-08-10,Parasite,2024-11-25,The Shawshank Redemption,44,40


In [3]:
df = db.query("""
    with movies as (
        select
            t1.id as user_id,
            t1.created_at,

            min(case when t2.status = 'finished' then t2.activity_date end) as first_date_finished,
            max(case when t2.status = 'finished' then t2.activity_date end) as last_date_finished,

            count(case when t2.status = 'started' then 1 end) as movies_started,
            count(case when t2.status = 'finished' then 1 end) as movies_finished

        from read_csv_auto('../Dataset/users_movies.csv') t1
        left join read_csv_auto('../Dataset/movie_activity.csv') t2
            on t1.id = t2.user_id
        group by 1,2
    )
        
    
    select
        *
    from movies
    where movies_finished >= 10
              
              
              """).to_df()

df

,user_id,created_at,first_date_finished,last_date_finished,movies_started,movies_finished
0,26,2024-03-27,2024-03-30,2024-10-08,42,19
1,27,2024-12-19,2025-01-13,2025-04-27,34,16
2,51,2023-10-29,2023-12-01,2025-04-26,42,28
3,25,2024-06-07,2024-11-03,2024-11-28,24,17
4,30,2023-10-04,2023-10-18,2024-08-22,50,17
...,...,...,...,...,...,...
493,924,2024-01-05,2024-07-21,2024-08-27,18,16
494,941,2024-08-18,2024-10-09,2024-10-30,36,10
495,957,2023-05-22,2023-10-28,2024-05-25,23,12
496,958,2024-09-05,2024-10-28,2025-02-16,22,21
